# A Jupyter Notebook to generate a CellML file for the selected structures in the Generic Cell Flatmap


In [1]:
### RUN THIS CELL IF IT'S THE FIRST TIME USING IPYWIDGETS & VOILA
# !jupyter nbextension enable --py widgetsnbextension --sys-prefix
# !jupyter serverextension enable voila --sys-prefix

In [2]:
# Libraries

import warnings
warnings.filterwarnings('ignore')

# b) Data extraction
import os, base64
import sys
import xml.etree.ElementTree as ET
import rdflib
import re
from lxml import etree

# c) General
import copy
import difflib
import numpy as np
import pandas as pd
import math
import operator as op
import ast
import re
import string
import csv
import xlsxwriter
import json
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive, IntSlider, FloatSlider
from IPython.display import display, clear_output, Image, FileLink, HTML
from SPARQLWrapper import SPARQLWrapper, JSON

# d) Plot
import matplotlib.pyplot as plt
from matplotlib import markers
import matplotlib.font_manager as font_manager
import matplotlib.colors
from pylab import rcParams
import matplotlib.image as mpimg

In [3]:
# Retreiving the annotations from the CellML parameter files
def getAnnotations(add):
    parser = etree.XMLParser(recover=True)
    root = etree.parse(add, parser).getroot()
    rdfGraph = rdflib.Graph()
    for rdfElement in root.iter():
        if rdfElement.tag.endswith('RDF'):
            try:
                rdfGraph.parse(data = etree.tostring(rdfElement), format="application/rdf+xml")
            except:
                pass
            
    def getLeaves(sbj, graph):
        triples = list(graph.triples((sbj,None,None)))
        leaves = []
        if len(triples)>0:
            for s, p, o in triples:
                result =  getLeaves(o,graph)
                leaves += result
            return list(set(leaves))
        else:
            return [sbj]
        
    # Opens the CellML file and returns the list of variables names
    f = open(add,'r')
    text = f.read()
    root = ET.fromstring(text)

    rdfs = root.findall('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}RDF')

    List1=[]
    for child in rdfs:
        for grand in child:
            List1.append(grand.attrib.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}about'))
    
    List = list(dict.fromkeys(List1))
    if '#metaid0' in List:
        List.remove('#metaid0')
        
        
    triplesList={}
    for i in range(len(List)):
        sbj = rdflib.URIRef(List[i])
        triplesList[i]=getLeaves(sbj, rdfGraph)


    rdfs=[]
    
    for item in triplesList:
        d = []
        for k in range(len(triplesList[item])):
            if 'opb' in triplesList[item][k] or 'chebi' in triplesList[item][k] or 'fma' in triplesList[item][k] or 'go' in triplesList[item][k]: 
                lSplit= triplesList[item][k].split('/')
                d.append(lSplit[-1])
            else: # free-style descriptions!
                d.append(str(triplesList[item][k]))
    
        rdfs.append(d)

    for child in root:
        if child.tag == '{http://www.cellml.org/cellml/1.1#}'+'component':
            modelComponentName = child.attrib.get('name')
    
    
    listP = []
    for item in List:
        for x in item.split('.'):
            if '#'+modelComponentName == x:
                itemP = item.replace(x+'.', '')
                listP.append(itemP)
                

            
    return [listP,rdfs,root]

In [4]:
def valExtraction(root,List): 
    
    init=[]; variables=[]; els=[]
    
    for child in root:
        if child.tag == '{http://www.cellml.org/cellml/1.1#}'+'component':
            modelComponentName = child.attrib.get('name')
            
    components = root.findall('{http://www.cellml.org/cellml/1.1#}component')
    
    for comp in components:
        variables.append(comp.findall('{http://www.cellml.org/cellml/1.1#}variable'))
        
    for element in List:  
        for var in variables:
            for v in var:   
                if modelComponentName+'.'+element == v.attrib['{http://www.cellml.org/metadata/1.0#}id']:
                    if 'initial_value' in v.attrib: # if any initial value exists take it
                        init.append(v.attrib['initial_value'])    
                    else:
                        init.append(None)
    return init

In [5]:
def checkMutual(parametersData):

    repeat = []
    mutualVars={}
    commonAnnotsDiffVals = {}; CommonAmountsAA = {}; CommonAmountsBB = {};
    for k1 in range(len(list(parametersData))):
        for k2 in range(len(list(parametersData))):
            if list(parametersData)[k1] != list(parametersData)[k2] and (list(parametersData)[k1],list(parametersData)[k2]) not in repeat and (list(parametersData)[k2],list(parametersData)[k1]) not in repeat:    
                if (list(parametersData.keys())[k1],list(parametersData.keys())[k2]) not in mutualVars:
                    mutualVars[(list(parametersData.keys())[k1],list(parametersData.keys())[k2])]=[]

                for i1 in range(len(list(parametersData.values())[k1])):
                    for i2 in range(len(list(parametersData.values())[k2])):
                        if all(elem in list(parametersData.values())[k1][i1][1] for elem in list(parametersData.values())[k2][i2][1]):
                            repeat.append((list(parametersData)[k1],list(parametersData)[k2]))

                            mutualVars[(list(parametersData.keys())[k1],list(parametersData.keys())[k2])].append(list(parametersData.values())[k1][i1][0])
                            mutualVars[(list(parametersData.keys())[k1],list(parametersData.keys())[k2])].append(list(parametersData.values())[k1][i1][1])

                            if list(parametersData.values())[k1][i1][2] != list(parametersData.values())[k2][i2][2]:

                                if (list(parametersData.keys())[k1],list(parametersData.keys())[k2]) not in commonAnnotsDiffVals:
                                    commonAnnotsDiffVals[(list(parametersData.keys())[k1],list(parametersData.keys())[k2])]=[]
                                    CommonAmountsAA[(list(parametersData.keys())[k1],list(parametersData.keys())[k2])] = []
                                    CommonAmountsBB[(list(parametersData.keys())[k1],list(parametersData.keys())[k2])] = []

                                commonAnnotsDiffVals[(list(parametersData.keys())[k1],list(parametersData.keys())[k2])].append(list(parametersData.values())[k1][i1][1])
                                CommonAmountsAA[(list(parametersData.keys())[k1],list(parametersData.keys())[k2])].append(list(parametersData.values())[k1][i1][2])
                                CommonAmountsBB[(list(parametersData.keys())[k1],list(parametersData.keys())[k2])].append(list(parametersData.values())[k2][i2][2])

    # Getting the preferred value from the user for the recognised same variables with different values
    for i in range(len(list(commonAnnotsDiffVals.keys()))):
        for j in range(len(list(commonAnnotsDiffVals.values())[i])):
            print('Different values found for: ')
            print('\n')
            print(list(commonAnnotsDiffVals.values())[i][j] ,list(commonAnnotsDiffVals.keys())[i],':',list(CommonAmountsAA.values())[i][j],  'and' , list(CommonAmountsBB.values())[i][j])   
            list(CommonAmountsAA.values())[i][j]=copy.deepcopy(input('Enter the preferred value:'))


    # Copying the changed values to the list of initial amounts of the model    
    for mutualTransporters in commonAnnotsDiffVals: # (x,y)
        for val1,i in zip(commonAnnotsDiffVals[mutualTransporters],range(len(commonAnnotsDiffVals[mutualTransporters]))):
            for transporterName in parametersData: 
                for val2 in parametersData[transporterName]:
                    if transporterName == mutualTransporters[0] and all(elem in val1 for elem in val2[1]):
                        val2[2] = copy.deepcopy(CommonAmountsAA[mutualTransporters][i])
                    if transporterName == mutualTransporters[1] and all(elem in val1 for elem in val2[1]):
                        val2[2] = copy.deepcopy(CommonAmountsAA[mutualTransporters][i])

    return [parametersData,mutualVars]               

In [6]:
 def textFileGen(speciesNoDuplicate,parametersData,speciesConstants,reaction_reactants,V,N,reaction_products):
    # Create a text file (TEXT)

    unitsImport = ['def model composed_model as',
                   '\tdef import using "units_BG.cellml" for',
                    '\t\tunit concentrationUnit using unit concentrationUnit;',
                   '\t\tunit speciesConstantUnit using unit speciesConstantUnit;',
                   '\t\tunit fluxUnit using unit fluxUnit;',
                    '\tenddef;',
                  '\tdef comp main as']

    with open('GFG.txt', 'w') as fp:
        pass    
    with open('GFG.txt', 'a') as file:
        for line in unitsImport:
            file.write(line+'\n')


    with open('GFG.txt', 'a') as file:
        file.write('\t\tvar t: second {init: 0};'+'\n')

    cellmlRef=[]        

    q = ['q_{0}'.format(x) for x in range(1000)]
    k=0
    checkDup=[]
    for species,i in zip(speciesNoDuplicate,range(len(speciesNoDuplicate))):
            for transporterName,annotation in parametersData.items():
                for value in annotation:
                    if all(el in value[1] for el in species):
                        flag=0; 
                        for j in range(len(checkDup)):
                            if all(x in checkDup[j] for x in value[1]):
                                flag=1

                        if flag==0:
                            checkDup.append(value[1])
                            cellmlRef.append((value[1],q[k],'concentrationUnit'))
                            with open('GFG.txt', 'a') as file:
                                file.write('\t\tvar ' +q[k]+': '+'concentrationUnit '+'{'+'init: '+value[2]+'}'+';'+'\n')
                                k+=1

    K = ['K_{0}'.format(x) for x in range(1000)]
    k=0
    checkDup=[]
    for constant,i in zip(speciesConstants,range(len(speciesConstants))):
        for transporterName,annotation in parametersData.items():
            for value in annotation:
                if all(el in value[1] for el in constant):
                    flag=0; 
                    for j in range(len(checkDup)):
                        if all(x in checkDup[j] for x in value[1]):
                            flag=1

                    if flag==0:
                        checkDup.append(value[1])
                        cellmlRef.append((value[1],K[k],'speciesConstantUnit'))
                        with open('GFG.txt', 'a') as file:
                            file.write('\t\tvar '+K[k]+': '+'speciesConstantUnit '+'{'+'init: '+value[2]+'}'+';'+'\n')
                            k+=1     


    v = ['v_{0}'.format(x) for x in range(1000)]
    k=0
    for flux in reaction_reactants:
        cellmlRef.append((flux,v[k],'fluxUnit'))
        with open('GFG.txt', 'a') as file:
            file.write('\t\tvar '+v[k]+': '+'fluxUnit '+';'+'\n')
            k+=1


    kappa = ['kappa_{0}'.format(x) for x in range(1000)]
    k=0
    for flux,i in zip(V,range(len(V))): 
        for transporterName,annotation in parametersData.items():
            for value in annotation:
                if flux in value[1]:
                    cellmlRef.append((value[1],kappa[k],'fluxUnit'))
                    with open('GFG.txt', 'a') as file:
                        file.write('\t\tvar '+ kappa[k]+': '+'fluxUnit '+'{'+'init: '+value[2]+'}'+';'+'\n')
                        k+=1

    # Generating the ODEs       
    with open('GFG.txt', 'a') as file:
        file.write('\n')

    for i in range(np.shape(N)[0]):
        for annotation,ID,unit in cellmlRef:
            if all(ele in speciesNoDuplicate[i] for ele in annotation):
                with open('GFG.txt', 'a') as file:
                    file.write('\t\tode('+ID+',t)=')
                for j in range(np.shape(N)[1]):  

                    if N[i][j] != 0:

                        if N[i][j] > 0:
                            for annotation2,ID2,unit2 in cellmlRef:
                                if V[j] == annotation2:
                                    with open('GFG.txt', 'a') as file:
                                        file.write('+'+str(N[i][j])+'{dimensionless}*'+ID2)

                    if N[i][j] < 0:
                        for annotation2,ID2,unit2 in cellmlRef:
                            if V[j] == annotation2:
                                with open('GFG.txt', 'a') as file:
                                    file.write(str(N[i][j])+'{dimensionless}*'+ID2)

        with open('GFG.txt', 'a') as file:
            file.write(';\n')


    # Generating the reaction rates
    with open('GFG.txt', 'a') as file:
        file.write('\n')

    for flux in V:
        for annotation1,ID1,unit1 in cellmlRef:
            if flux == annotation1:
                with open('GFG.txt', 'a') as file:
                    file.write('\t\t '+ ID1+'= ') 
                for annotation2,ID2,unit2 in cellmlRef:
                    if all(el in annotation2  for el in [flux]+['OPB_01296']):
                        with open('GFG.txt', 'a') as file:
                            file.write(ID2+'*(') 
        length = 0                    
        for stoi,reactant in reaction_reactants[flux]:
            length+=1
            for annotation,ID,unit in cellmlRef:
                if all(el in reactant for el in annotation):
                    Q = copy.deepcopy(annotation)
                    Q.remove('OPB_00340')
                    for annotation2,ID2,unit2 in cellmlRef:
                        if all(el in annotation2 for el in Q+['OPB_00079']):
                            with open('GFG.txt', 'a') as file:
                                file.write('pow('+ID+'*'+ID2+','+str(stoi)+'{dimensionless}'+')') 

            if length < len(reaction_reactants[flux]):
                with open('GFG.txt', 'a') as file:
                    file.write(' * ')


        with open('GFG.txt', 'a') as file:
            file.write(' - ')

        length = 0 
        for stoi,product in reaction_products[flux]:
            length+=1
            for annotation,ID,unit in cellmlRef:
                if all(el in product for el in annotation):
                    Q = copy.deepcopy(annotation)
                    Q.remove('OPB_00340')
                    for annotation2,ID2,unit2 in cellmlRef:
                        if all(el in Q+['OPB_00079'] for el in annotation2):
                            with open('GFG.txt', 'a') as file:
                                file.write('pow('+ID+'*'+ID2+','+str(stoi)+'{dimensionless}'+')') 

            if length < len(reaction_products[flux]):
                with open('GFG.txt', 'a') as file:
                    file.write(' * ')




        with open('GFG.txt', 'a') as file:
            file.write(');\n')





    with open('GFG.txt', 'a') as file:
        file.write('\tenddef;'+'\n'+'enddef;')
        
    return cellmlRef

In [7]:
def pmrSearching(name):
    sparqlendpoint = 'https://models.physiomeproject.org/pmr2_virtuoso_search'
    sparql = SPARQLWrapper(sparqlendpoint)

    def search_entity(terms):
        query = """SELECT ?graph ?Model_entity WHERE {{ GRAPH ?graph {{ ?Model_entity ?p ?o FILTER REGEX(LCASE(STR(?Model_entity)), '{terms}')}}}}""".format(terms=terms)
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        graphs = sparql.query().convert()
        return graphs

    def search_model(terms):
        terms = terms.lower()
        entities = search_entity(terms)
        model = set()
        for entity in entities['results']['bindings']:
            workspace = entity['graph']['value']
            cellml = entity['Model_entity']['value'].split('#')[0]
            if not cellml.startswith('http') and terms in cellml.lower():
                model.update([workspace+'/rawfile/HEAD/'+cellml])
        return list(model)

    pmrModel = search_model(name)

    return pmrModel

In [8]:
def BGmodelBuilder(parametersData,inUse):
    reaction_reactants={}
    reaction_products={}
    for transporterName in parametersData:  
        for entity in parametersData[transporterName]:
            for reaction in Reactions[inUse[transporterName][0]]:
                if transporterName+'_'+reaction in entity[1]:
                    reaction_reactants[transporterName+'_'+reaction]=[]
                    reaction_products[transporterName+'_'+reaction]=[]
                    for stoi in reaction_reactants_templates[inUse[transporterName][0]]:
                        for key,values in stoi.items():
                            if key == reaction:
                                for value in values: # (value[0],value[1])=(1,Ai)
                                    for reference in referenceAnnotations[inUse[transporterName][0]]:
                                        for ID in reference:
                                            for annot in reference[ID]:
                                                if ID == value[1]: 
                                                    for entity0 in parametersData[transporterName]:
                                                        if all(element in annot for element in entity0[1]):
                                                            X=copy.deepcopy(annot)
                                                            X.append(transporterName)
                                                            reaction_reactants[transporterName+'_'+reaction].append((value[0],X))
                    for stoi in reaction_products_templates[inUse[transporterName][0]]:
                        for key,values in stoi.items():
                            if key == reaction:
                                for value in values: # (value[0],value[1])=(1,Ai)
                                    for reference in referenceAnnotations[inUse[transporterName][0]]:
                                        for ID in reference:
                                            for annot in reference[ID]:
                                                if ID == value[1]: 
                                                    for entity0 in parametersData[transporterName]:
                                                        if all(element in annot for element in entity0[1]):
                                                            X=copy.deepcopy(annot)
                                                            X.append(transporterName)
                                                            reaction_products[transporterName+'_'+reaction].append((value[0],X))                                           

    V = []
    for flux in reaction_reactants:
        V.append(flux)

    Species = []
    for transporterName in parametersData:
        for entity in parametersData[transporterName]:
            if 'OPB_00340' in entity[1]:
                X = copy.deepcopy(entity[1])
                X.append(transporterName)
                Species.append(X)

    speciesNoDuplicate = []
    for s in Species:
        for transporterName in inUse:
            if transporterName in s:
                ss = copy.deepcopy(s)
                ss.remove(transporterName)
                if ss not in speciesNoDuplicate:
                    speciesNoDuplicate.append(ss)

    speciesConstants=[]
    for species in speciesNoDuplicate:
        X = copy.deepcopy(species)
        X.remove('OPB_00340')
        for transporterName in parametersData:
            for entity in parametersData[transporterName]:
                if all(el in entity[1] for el in ['OPB_00079']+X) and entity[1] not in speciesConstants:
                    speciesConstants.append(entity[1])


    M=np.zeros((len(Species),len(V)))
    for s in range(len(Species)):
        for key,i in zip(reaction_products,range(len(reaction_products))):
            for stoi,val  in reaction_products[key]:
                if all(el in val for el in Species[s]):
                    M[s][i] = stoi
        for key,i in zip(reaction_reactants,range(len(reaction_reactants))):
            for stoi,val in reaction_reactants[key]:
                if all(el in val for el in Species[s]):
                    M[s][i] = -stoi


    N=np.zeros((len(speciesNoDuplicate),len(V)))
    for transporterName in inUse:
        for v,i in zip(V,range(len(V))):
            if transporterName in v:
                for s1 in range(len(speciesNoDuplicate)):
                    for s2 in range(len(Species)):
                        if transporterName in Species[s2]:
                            if all(el in Species[s2] for el in speciesNoDuplicate[s1]):
                                N[s1][i] = M[s2][i]
                                
    return [N,V,speciesConstants,speciesNoDuplicate,reaction_reactants,reaction_products]


In [9]:
def pmrRDFsearch(terms):
    
    sparqlendpoint = 'https://models.physiomeproject.org/pmr2_virtuoso_search'
    sparql = SPARQLWrapper(sparqlendpoint)

    sparql_tmpl = '''SELECT DISTINCT ?g ?v WHERE {{ GRAPH ?g {{
    ?v ?p1 [?p2 ?b; ?p3 [?p4 ?d; ?p5 [?p6 ?f]]] .
    FILTER (regex(str(?b),"{opb}") && regex(str(?d),"{chebi}") && regex(str(?f),"{fma}"))
    }} }}'''

    for c in terms:
        if c.startswith('CHEBI'):
            chebi = c
        elif c.startswith('OPB'):
            opb = c
        elif c.startswith('FMA'):
            fma = c
    query = sparql_tmpl.format(opb=opb, chebi=chebi, fma=fma)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    rst =  sparql.query().convert()
    if len(rst['results']['bindings']) > 0:
        cellmls = {}
        for r in rst['results']['bindings']:
            cellml_file, cellml_var = r['v']['value'].split('#')
            cellml_path = r['g']['value'] + '/rawfile/HEAD/' + cellml_file
            if cellml_path not in cellmls:
                cellmls[cellml_path] = []
            cellmls[cellml_path] += [cellml_var]
        return cellmls
    return {}


In [10]:
def minMaxRange(cellmlRef):
    valueRange = {}
    valueMinMax = {}

    PMRmodelsAddress = '/Users/nsha457/Documents/Jupyter_files/CellmlScriptProduction/Exemplar models on PMR'
    for file in os.listdir(PMRmodelsAddress):
        if file.endswith(".cellml"):
            [List,rdfs,root] = getAnnotations(PMRmodelsAddress+'/{0}'.format(file))
            initialVals = valExtraction(root,List)
            for i in range(len(cellmlRef)):
                for j in range(len(rdfs)):
                    if all(ele in cellmlRef[i][0] for ele in rdfs[j]):
                        if cellmlRef[i][1] in valueRange.keys():
                            valueRange[cellmlRef[i][1]].append(initialVals[j])
                        else:
                            valueRange[cellmlRef[i][1]]=[]
                            valueRange[cellmlRef[i][1]].append(initialVals[j])   

    for key,value in valueRange.items():
        valueMinMax[key] = []
        valueMinMax[key].append(min(valueRange[key]))
        valueMinMax[key].append(max(valueRange[key]))
        
    return valueMinMax

In [11]:
# Here we see which group of transporters/channels belong to which template
templateGroups = {'SLC2':['SLC2A1','SLC2A2','SLC2A3'],
                  'SLC4':['SLC4A1','SLC4A2','SLC4A3']}

In [12]:
referenceAnnotations = {'SLC2': [{'Ai':[['FMA:226050', 'OPB_00340', 'CHEBI:17234'],['FMA:226050', 'OPB_00340', 'CHEBI:28260'],['FMA:226050', 'OPB_00340', 'CHEBI:37684'],['FMA:226050', 'OPB_00340', 'CHEBI:5417']]},
                                {'Ao':[['OPB_00340', 'FMA:70022', 'CHEBI:17234'], ['OPB_00340', 'FMA:70022', 'CHEBI:28260'],['OPB_00340', 'FMA:70022', 'CHEBI:37684'],['OPB_00340', 'FMA:70022', 'CHEBI:5417']]},
                                {'Ei':[['FMA:226050', 'OPB_00340', 'CHEBI:38807']]},
                                {'Eo':[['OPB_00340', 'FMA:70022', 'CHEBI:38807']]},
                                {'EAi':[['CHEBI:81851', 'FMA:226050', 'OPB_00340']]},                              
                                {'EAo':[['CHEBI:81851', 'OPB_00340', 'FMA:70022']]},
                                {'K_Ai':[['OPB_00079', 'FMA:226050', 'CHEBI:17234'], ['OPB_00079', 'FMA:226050', 'CHEBI:28260'], ['OPB_00079', 'FMA:226050', 'CHEBI:37684'], ['OPB_00079', 'FMA:226050', 'CHEBI:5417']]},
                                {'K_Ao':[['OPB_00079', 'FMA:70022', 'CHEBI:17234'], ['OPB_00079', 'FMA:70022', 'CHEBI:28260'], ['OPB_00079', 'FMA:70022', 'CHEBI:37684'], ['OPB_00079', 'FMA:70022', 'CHEBI:5417']]},
                                {'K_Ei':[['OPB_00079', 'FMA:226050', 'CHEBI:38807']]},
                                {'K_Eo':[['OPB_00079', 'FMA:70022', 'CHEBI:38807']]},
                                {'K_EAi':[['CHEBI:81851', 'FMA:226050', 'OPB_00079']]},
                                {'K_EAo':[['CHEBI:81851', 'FMA:70022', 'OPB_00079']]},
                                {'kappa_Re1':[['CHEBI:64297', 'SLC2A1_Re1', 'OPB_01296']]},
                                {'kappa_Re2':[['SLC2A1_Re2', 'CHEBI:64297', 'OPB_01296']]},
                                {'kappa_Re3':[['CHEBI:64297', 'SLC2A1_Re3', 'OPB_01296']]},
                                {'kappa_Re4':[['SLC2A1_Re4', 'CHEBI:64297', 'OPB_01296']]},],
                       'SLC4': [{'Ai':[['FMA:226050', 'OPB_00340', 'CHEBI:17234'],["randomRDF17"]]},
                                {'Ao':[['OPB_00340', 'FMA:70022', 'CHEBI:17234'], ["randomRDF18"]]},
                                {'Ei':[['CHEBI:39000', 'OPB_00340', 'FMA:226050'], ["randomRDF19"]]},
                                {'Eo':[['CHEBI:39000', 'OPB_00340', 'FMA:70022'], ["randomRDF20"]]},
                                {'EAi':[['CHEBI:31340', 'FMA:226050', 'OPB_00340'], ["randomRDF21"]]},
                                {'EAo':[['CHEBI:31340', 'FMA:70022', 'OPB_00340'], ["randomRDF22"]]},
                                {'Bi':[['OPB_00340', 'FMA:226050', 'CHEBI:29101'], ["randomRDF23"]]},
                                {'Bo':[['OPB_00340', 'FMA:70022', 'CHEBI:29101'], ["randomRDF24"]]},
                                {'EBi':[['OPB_00340', 'FMA:226050', 'CHEBI:62965'], ["randomRDF25"]]},
                                {'EBo':[['OPB_00340', 'FMA:70022', 'CHEBI:62965'], ["randomRDF26"]]},
                                {'K_Ai':[['FMA:226050', 'CHEBI:17234', 'OPB_00079'],["randomRDF27"]]},
                                {'K_Ao':[['OPB_00079', 'FMA:70022', 'CHEBI:17234'], ["randomRDF28"]]},
                                {'K_Ei':[['CHEBI:39000', 'OPB_00079', 'FMA:226050'], ["randomRDF29"]]},
                                {'K_Eo':[['CHEBI:39000', 'OPB_00079', 'FMA:70022'], ["randomRDF30"]]},
                                {'K_EAi':[['CHEBI:31340', 'OPB_00079', 'FMA:226050'], ["randomRDF31"]]},
                                {'K_EAo':[['CHEBI:31340', 'OPB_00079', 'FMA:70022'], ["randomRDF32"]]},
                                {'K_Bi':[['OPB_00079', 'FMA:226050', 'CHEBI:29101'], ["randomRDF33"]]},
                                {'K_Bo':[['OPB_00079', 'FMA:70022', 'CHEBI:29101'], ["randomRDF34"]]},
                                {'K_EBi':[['OPB_00079', 'FMA:226050', 'CHEBI:62965'], ["randomRDF35"]]},
                                {'K_EBo':[['OPB_00079', 'FMA:70022', 'CHEBI:62965'], ["randomRDF36"]]},
                                {'kappa_Re1':[['CHEBI:64297', 'SLC4A1_Re1', 'OPB_01296'], ["randomRDF37"]]},
                                {'kappa_Re2':[['CHEBI:64297', 'SLC4A1_Re2', 'OPB_01296'], ["randomRDF38"]]},
                                {'kappa_Re3':[['CHEBI:64297', 'SLC4A1_Re3', 'OPB_01296'], ["randomRDF39"]]},
                                {'kappa_Re4':[['CHEBI:64297', 'SLC4A1_Re4', 'OPB_01296'], ["randomRDF40"]]},
                                {'kappa_Re5':[['CHEBI:64297', 'SLC4A1_Re5', 'OPB_01296'], ["randomRDF41"]]},
                                {'kappa_Re6':[['CHEBI:64297', 'SLC4A1_Re6', 'OPB_01296'], ["randomRDF42"]]},
                               ]}

In [13]:
Reactions={'SLC2': ['Re1', 'Re2', 'Re3', 'Re4'],
           'SLC4': ['Re1', 'Re2', 'Re3', 'Re4', 'Re5', 'Re6']}

reaction_reactants_templates={'SLC2': [{'Re1':[(1,'EAi')]}, {'Re2':[(1,'Eo'),(1,'Ao')]}, {'Re3':[(1,'Ei')]}, {'Re4':[(1,'EAo')]}],
                              'SLC4': [{'Re1':[(1,'Ao'),(1,'Eo')]}, {'Re2':[(1,'EAi')]}, {'Re3':[(1,'EBo')]}, {'Re4':[(1,'Bi'),(1,'Ei')]}, {'Re5':[(1,'EBi')]}, {'Re6':[(1,'EAo')]}]   }

reaction_products_templates={'SLC2': [{'Re1':[(1,'Ai'),(1,'Ei')]}, {'Re2':[(1,'EAo')]}, {'Re3':[(1,'Eo')]}, {'Re4':[(1,'EAi')]}],
                             'SLC4': [{'Re1':[(1,'EAo')]}, {'Re2':[(1,'Ai'),(1,'Ei')]}, {'Re3':[(1,'Bo'),(1,'Eo')]}, {'Re4':[(1,'EBi')]}, {'Re5':[(1,'EBo')]}, {'Re6':[(1,'EAi')]}]}


## Section 1: Model selection

In [21]:

channelSelection1 = widgets.Dropdown(
            options=['SLC2A1', 'SLC4A1'],
            description='Select your first model:',
    style={'description_width': 'initial'},
            disabled=False
        )

channelSelection2 = widgets.Dropdown(
            options=['SLC2A1', 'SLC4A1'],
            description='Select your second model:',
    style={'description_width': 'initial'},
            disabled=False
        )
       
button_search = widgets.Button(
                description='Search on PMR',
                tooltip='Search',
                style={'description_width': 'initial'}
            )

output = widgets.Output(layout={'border': '1px solid black'})
def on_button_clicked1(event):
    with output:
        clear_output()
        channelSelected = [channelSelection1.value,channelSelection2.value]
        inUse={}
        for selected in channelSelected:
            for key in templateGroups.keys():
                for transporterName,i in zip(templateGroups[key],range(len(templateGroups[key]))):
                    if selected == transporterName:
                        inUse[selected]=[]
                        inUse[transporterName].append(key)
                        
        pmrModel1 = pmrSearching(channelSelection1.value)
        pmrModel2 = pmrSearching(channelSelection2.value)
        
        print(f"The first TEMPLATE is {inUse[channelSelection1.value][0]} with the link to the parameterset: {pmrModel1[0]}")
        print(f"The second TEMPLATE is {inUse[channelSelection2.value][0]} with the link to the parameterset: {pmrModel2[0]}")
        
        
            
button_search.on_click(on_button_clicked1)
vbox_result = widgets.VBox([button_search, output])   

vbox_text = widgets.VBox([channelSelection1, channelSelection2, vbox_result])
page1 = widgets.HBox([vbox_text])

display(page1)



## Section 2: The Steady-states (SS)


In [22]:

button_properties = widgets.Button(
                description='Show SS',
                tooltip='Description',
                style={'description_width': 'initial'}
            )
output2 = widgets.Output(layout={'border': '1px solid black'})

def on_button_clicked2(event):
    with output2:
        clear_output()
        
        channelSelected = [channelSelection1.value,channelSelection2.value]
        inUse={}
        for selected in channelSelected:
            for key in templateGroups.keys():
                for transporterName,i in zip(templateGroups[key],range(len(templateGroups[key]))):
                    if selected == transporterName:
                        inUse[selected]=[]
                        inUse[transporterName].append(key)

        annotations={}
        for transporterName in inUse:
            addressP = '/Users/nsha457/Documents/Jupyter_files/CellmlScriptProduction/enzymes/{0}.cellml'.format(transporterName)
            [List,rdfs,root] = getAnnotations(addressP)
            initialVals = valExtraction(root,List)
            annotations[transporterName] = []
            annotations[transporterName] = [List,rdfs,initialVals]

        parametersData = {}
        for key in annotations:
            parametersData[key] = []
            for j in range(len(annotations[key][0])):
                parametersData[key].append([annotations[key][0][j],annotations[key][1][j],annotations[key][2][j]])


        refInUse = {'SLC2A1': ['GLUT2'], 'SLC4A1': ['GLUT4']}

        annotations_ref={}
        df_RefModels = {}
        for transporterName in refInUse:
            addressRefModels = './{0}.cellml'.format(refInUse[transporterName][0])
            addressRefData = './{0}.csv'.format(refInUse[transporterName][0])

            [List_ref,rdfs_ref,root_ref] = getAnnotations(addressRefModels)
            initialVals_ref = valExtraction(root_ref,List_ref)
            annotations_ref[transporterName] = []
            annotations_ref[transporterName] = [List_ref,rdfs_ref,initialVals_ref]

            df_RefModels[transporterName] = pd.read_csv(addressRefData)

        parametersData_ref = {}
        for key in annotations_ref:
            parametersData_ref[key] = []
            for j in range(len(annotations_ref[key][0])):
                parametersData_ref[key].append([annotations_ref[key][0][j],annotations_ref[key][1][j],annotations_ref[key][2][j]])


        SS = {}  
        pmrModelSimilarRDF={}
        for key,values in parametersData.items():
            SS[key]={}
            for value in values:
                for value_ref in parametersData_ref[key]:
                    if all(el in value[1] for el in value_ref[1]):

                        SS[key][value_ref[0]] = df_RefModels[key][[x for x in df_RefModels[key].columns if x == value_ref[0]]].values[-1][0]
#                         pmrModelSimilarRDF[value[0]] = pmrRDFsearch(value[1])
                        

        text_0 = widgets.HTML(value="<h5><b>According to PMR, your selected protein {} should have the following SS values:<b><h5>".format(channelSelection1.value))
        display(text_0)
#         text_00 = widgets.HTML(value="<h5><b>Models are:<b><h5>")
#         display(text_00)
#         print(f"{list(pmrModelSimilarRDF.values())}")

        print(f"{SS[channelSelection1.value]}\n")
        text_1 = widgets.HTML(value="<h5><b>According to PMR, your selected protein {} should have the following SS values:<b><h5>".format(channelSelection2.value))
        display(text_1)
        print(f"{SS[channelSelection2.value]}\n")
                
button_properties.on_click(on_button_clicked2)
vbox_result = widgets.VBox([button_properties, output2]) 

page2 = widgets.HBox([vbox_result])

display(page2)


## Section 3: Model composition


In [223]:
button_compose = widgets.Button(
                description='Compose models',
                tooltip='Description',
                style={'description_width': 'initial'}
            )
output3 = widgets.Output(layout={'border': '1px solid black'})

def on_button_clicked3(event):
    with output3:
#         clear_output()
        
        channelSelected = [channelSelection1.value,channelSelection2.value]
        inUse={}
        for selected in channelSelected:
            for key in templateGroups.keys():
                for transporterName,i in zip(templateGroups[key],range(len(templateGroups[key]))):
                    if selected == transporterName:
                        inUse[selected]=[]
                        inUse[transporterName].append(key)

        annotations={}
        for transporterName in inUse:
            addressP = '/Users/nsha457/Documents/Jupyter_files/CellmlScriptProduction/enzymes/{0}.cellml'.format(transporterName)
            [List,rdfs,root] = getAnnotations(addressP)
            initialVals = valExtraction(root,List)
            annotations[transporterName] = []
            annotations[transporterName] = [List,rdfs,initialVals]

        parametersData = {}
        for key in annotations:
            parametersData[key] = []
            for j in range(len(annotations[key][0])):
                parametersData[key].append([annotations[key][0][j],annotations[key][1][j],annotations[key][2][j]])


        refInUse = {'SLC2A1': ['GLUT2'], 'SLC4A1': ['GLUT4']}

        annotations_ref={}
        df_RefModels = {}
        for transporterName in refInUse:
            addressRefModels = './{0}.cellml'.format(refInUse[transporterName][0])
            addressRefData = './{0}.csv'.format(refInUse[transporterName][0])

            [List_ref,rdfs_ref,root_ref] = getAnnotations(addressRefModels)
            initialVals_ref = valExtraction(root_ref,List_ref)
            annotations_ref[transporterName] = []
            annotations_ref[transporterName] = [List_ref,rdfs_ref,initialVals_ref]

            df_RefModels[transporterName] = pd.read_csv(addressRefData)

        parametersData_ref = {}
        for key in annotations_ref:
            parametersData_ref[key] = []
            for j in range(len(annotations_ref[key][0])):
                parametersData_ref[key].append([annotations_ref[key][0][j],annotations_ref[key][1][j],annotations_ref[key][2][j]])


        SS = {}        
        for key,values in parametersData.items():
            SS[key]={}
            for value in values:
                for value_ref in parametersData_ref[key]:
                    if all(el in value[1] for el in value_ref[1]):

                        SS[key][value_ref[0]] = df_RefModels[key][[x for x in df_RefModels[key].columns if x == value_ref[0]]].values[-1][0]



        [parametersData,mutualVars] = checkMutual(parametersData)
        

        [N,V,speciesConstants,speciesNoDuplicate,reaction_reactants,reaction_products]=BGmodelBuilder(parametersData,inUse)
                                    
        cellmlRef = textFileGen(speciesNoDuplicate,parametersData,speciesConstants,reaction_reactants,V,N,reaction_products)
            
        valueMinMax = minMaxRange(cellmlRef)
        
        text_0 = widgets.HTML(value="<h3>We found the following ranges for the following parameters:<h3>")
        display(text_0)
#         print(valueMinMax,'\n')
        
        
            
        q_9=widgets.RadioButtons(options=[float(valueMinMax['q_9'][0])])
        K_9=widgets.RadioButtons(options=[float(valueMinMax['K_9'][0])]) 
        q_10=FloatSlider(min=float(valueMinMax['q_10'][0]),max=float(valueMinMax['q_10'][1]),step=0.001,layout=widgets.Layout(width='75%'), readout_format='.3f')
        K_10=FloatSlider(min=float(valueMinMax['K_10'][0]),max=float(valueMinMax['K_10'][1]),step=0.001,layout=widgets.Layout(width='75%'), readout_format='.3f')
        
        def plotf(q_9,K_9,q_10,K_10):
            plt.figure(figsize=(5,5))
            x = np.linspace(0, 100, num=100)
            plt.plot(x, np.ones(100)*(K_9*q_10*4*4*8-K_10*2*70)*1e-6,label='Glucose_i',color='r', linewidth=2) #SS relationship for q_9
            plt.plot(x, np.ones(100)*(K_10*30*3*2-2*2*K_9*q_9*8)*1e-6,label='Sodium_i',color='steelblue', linewidth=2) # SS relationship for q_10
            plt.grid(color='gray', linestyle='-', linewidth=0.5)
            plt.ylabel('Concentration')
            plt.title('Steady-state concentrations')
            plt.legend(loc='best')
            plt.show()
            return 
        
        interact(plotf, q_9=q_9, K_9=K_9, q_10=q_10, K_10=K_10)
        
        # Create the button widget
        button = widgets.Button(description='Save number')
        # Define a function to handle the button click event
        def save_number(button_click):
            selected_number = {'q_9':q_9.value,'K_9':K_9.value,'q_10':q_10.value,'K_10':K_10.value}
            with open('selected_number.txt', 'w') as f:
                f.write(str(selected_number))
            print(f'Number {selected_number} saved to file.')

        # Attach the save_number function to the button click event
        button.on_click(save_number)
        # Display the widgets
        display(button)


    return 

        

button_compose.on_click(on_button_clicked3)
vbox_result = widgets.VBox([button_compose, output3]) 
page3 = widgets.HBox([vbox_result])
display(page3)




In [216]:
q_9=widgets.RadioButtons()
display(q_9)

RadioButtons(options=(), value=None)

## Section 4: Model rendering

In [ ]:
button_cellml = widgets.Button(
                description='Generate code',
                tooltip='Description',
                style={'description_width': 'initial'}
            )
output4 = widgets.Output(layout={'border': '1px solid black'})

def on_button_clicked4(event):
    with output4:
        text_0 = widgets.HTML(value="<h3>The generated CellML code for the composition of SLC2A1 & SLC4A1 is:</h3>")
        vbox_text = widgets.VBox([text_0])
        display(vbox_text)
        with open('GFG.txt') as f:
            contents = f.read()
        print(contents)

button_cellml.on_click(on_button_clicked4)
vbox_result = widgets.VBox([button_cellml, output4]) 
display(vbox_result)
